__IMPORTANT__: This tutorial needs an update and is not currently working. 

# Annosaurus Tutorial

This python3 notebook demonstrates the usage of the [Annosaurus](https://github.com/underwatervideo/annosaurus) API which is used for creating and editing video annotations. To get started you will need to start annosaurus. If you have [Docker](https://www.docker.com/) installed you can spin up annosaurus for testing with:

```
docker run --name=anno -p 8080:8080 mbari/annosaurus
```


### Configure imports and define helper functions

In [1]:
import datetime
import json
import pprint
import random
import requests
import urllib
import uuid

def show(s, data = None):
    pp = pprint.PrettyPrinter(indent=2)
    print("--- " + s)
    if data:
      pp.pprint(data)
    
def iso8601():
    return datetime.datetime.now(datetime.timezone.utc).isoformat()[0:-6] + "Z"
    
def parse_response(r):
    try:
       return json.loads(r.text)
    except:
        s = "URL: %s\n%s (%s): %s" % (r.request.url, r.status_code, r.reason, r.text)
        print(s)
        return {}
    
    
def delete(url):
    return parse_response(requests.delete(url))

def get(url):
    return parse_response(requests.get(url))
    
def post(url, data = {}):
    return parse_response(requests.post(url, data))

def put(url, data = {}):
    return parse_response(requests.put(url, data))


### Define URLs

Normally in an app or script you just define the endpoint and build the other API urls from that.

In [2]:
# Define endpoint. 
endpoint = "http://10.0.1.251:8080"

annotation_url = "%s/v1/annotations" % (endpoint)
image_url = "%s/v1/images" % (endpoint)
image_reference_url = "%s/v1/imagereferences" % (endpoint)
imaged_moment_url = "%s/v1/imagedmoments" % (endpoint)
observation_url = "%s/v1/observations" % (endpoint)
association_url = "%s/v1/associations" % (endpoint)


# High-level APIs

The Annotation and Image APIs are high-level APIs to greatly simplify general usage. They are an abstraction for the lower level _ImagedMoment_, _Observation_, _ImageReference_, and _Association_ APIs. For creating and updating annotations and images, use the highly level APIs. If you need to delete or do fancy stuff you can use the lower level APIs

## Annotation API

The annotation API is used for creating and modifying annotations. You __can not delete__ with this API. Instead, you use the _Observation API_.

Note that the APIs use [UUIDs](https://en.wikipedia.org/wiki/Universally_unique_identifier) as keys to identify particular items. 

### Create

In [3]:
# id to a video file. Typically you get this from your video asset manager. 
# We're just creating a random one to use for this demo.
video_reference_uuid = str(uuid.uuid4())

# Create w/ minimum allowed fields
annotation = post(annotation_url,
                  data = {"video_reference_uuid": video_reference_uuid,
                          "concept": "Nanomia bijuga",
                          "observer": "brian",
                          "recorded_timestamp": "2016-07-28T14:29:01.030Z"})
show("POST: " + annotation_url, annotation)


ConnectionError: HTTPConnectionPool(host='10.0.1.251', port=8080): Max retries exceeded with url: /v1/annotations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f323bc694e0>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
# Create with all possible fields, including optional values
annotation = post(annotation_url,
                  data = {"video_reference_uuid": video_reference_uuid,      # video or image grouping
                          "concept": "Aegina citrea",                        # Name of what you saw
                          "observer": "schlin",                              # Who made the observation
                          "observation_timestamp": "2016-07-28T15:01:02Z",   # When the observation was make. Default is the servers timestamp
                          "timecode": "01:23:34:09",                         # A tape timecode of annotation
                          "elapsed_time_millis": "112345",                   # Time since start of video of annotation
                          "duration_millis": "1200",                         # How long was object observed
                          "group": "ROV",                                    # A logical group. At MBARI, we might use "ROV", "AUV", "Station M"
                          "activity": "transect",                            # Another logical group. At MBARI, we would use, ascent, descent, transect, cruise, etc.
                          "recorded_timestamp": "2016-07-28T14:39:02.123Z"}) # The time the frame was recorded. e.g. We saw this Aegina on this date.
show("POST: " + annotation_url, annotation)

### Update

In [ ]:
# Update/Modify an existing annotation

observation_uuid = annotation["observation_uuid"]

# At a minimum you need the observation_uuid and one field. The observation_timestamp
# will automatically be updated to the time on the server (UTC). Here we just change
# the concept name. Normally, you might need to update the observer field too.
url = "%s/%s" % (annotation_url, observation_uuid)
annotation = put(url,
                 data = {"observation_uuid": observation_uuid,
                         "concept": "Atolla"})
show("PUT: " + url, annotation)

In [ ]:
# You can update any and all fields in one call as we do here.
annotation = put(url,
                  data = {"video_reference_uuid": str(uuid.uuid4()),         # Here we move the annotation to a new video
                          "concept": "Pandalus platyceros",                  # Name of what you saw
                          "observer": "danelle",                             # Who made the observation
                          "observation_timestamp": iso8601(),                # When the observation was make. Default is the servers timestamp
                          "timecode": "08:00:34:09",                         # A tape timecode of annotation
                          "elapsed_time_millis": "3045999",                  # Time since start of video of annotation
                          "duration_millis": "8",                            # How long was object observed
                          "group": "AUV",                                    # A logical group. At MBARI, we might use "ROV", "AUV", "Station M"
                          "activity": "descent",                             # Another logical group. At MBARI, we would use, ascent, descent, transect, cruise, etc.
                          "recorded_timestamp": "2017-07-28T14:39:02.123Z"}) # The time the frame was recorded. e.g. We saw this Aegina on this date.
show("PUT: " + url, annotation)


### Find

In [ ]:
# Find an annotation by observation_uuid
url = "%s/%s" % (annotation_url, annotation["observation_uuid"])
annotation = get(url)
show("GET: " + url, annotation)

In [ ]:
# Find all annotation for a specific video
url = "%s/videoreference/%s" % (annotation_url, annotation["video_reference_uuid"])
annotations = get(url)
show("GET: " + url, annotations)

## Image API

The image API is for registering images. These image can be associated with a particular video but do not have to be. Images are referenced via a URL, not a file path.

### Create

In [ ]:
# Although called 'video_reference_uuid', it's a logical group for annotations. 
# For an image set this might just be a random value.
video_reference_uuid = annotation['video_reference_uuid']

# Minimum required fields. Note that one or more indexes are required. I use recorded_timestamp here, 
# but you could also use 'elapased_time_millis' or 'timecode'
image = post(image_url, data = {
    "video_reference_uuid": video_reference_uuid,
    "url": "http://foobar.org/awesomeimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": annotation['recorded_timestamp']})
show("POST:" + image_url, image)


In [ ]:
# Create with all fields
image = post(image_url, data = {
    "video_reference_uuid": video_reference_uuid,
    "url": "http://foobar.org/onfleekimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": iso8601(),
    "timecode": "01:23:45:09",
    "elapsed_time_millis": 123456,
    "width_pixels": 1920,
    "height_pixels": 1080,
    "format": "image/jpg",
    "description": "left-image"})
show("POST: " + image_url , image)

### Update

In [ ]:
# This changes all parameters, but you only need to include the ones you want to change
url = "%s/%s" % (image_url, image['image_reference_uuid'])
image = put(url, data = {
        "video_reference_uuid": str(uuid.uuid4()),
        "url": "http://foobar.org/yodaimage_" + str(random.randint(0, 100000)) + ".tif",
        "recorded_timestamp": iso8601(),
        "timecode": "02:00:15:19",
        "elapsed_time_millis": 666,
        "width_pixels": 2920,
        "height_pixels": 980,
        "format": "image/tiff",
        "description": "right-image"})
show("PUT: " + url, image) 

### Find

In [ ]:
# Find an image by image_reference_uuid
url = "%s/%s" %(image_url, image['image_reference_uuid'])
image = get(url)
show("GET: " + url, image)

In [ ]:
# Find all images for a given group (or video)
url = "%s/videoreference/%s" % (image_url, video_reference_uuid)
images = get(url)
show("GET: " + url, images)

In [ ]:
# Find image metadata by a URL. Note: you can not use the raw url, urlencode it first
encoded_url = urllib.parse.quote_plus(image['url'])
url = "%s/url/%s" % (image_url, encoded_url)
image = get(url)
show("GET: " + url, image)

# Low Level APIs

## API Diagram

![UML Class Diagram](https://github.com/underwatervideo/annosaurus/raw/master/src/site/images/annosaurus_classes.png)

## Association API

Associations are extra descriptions that you can attach to your annotation. Things like color, comments, resting upon, position in the image, a measurement, etc. Associations have the form 'link name | to concept | link value'. 

- link name: Tells you what the association contains. e.g. 'comment' or 'upon substrate'
- to concept: Indicates are relation. For example if your _link name_ is `eating` the _to concept_ indicates what it's eating. e.g. 'Squid'. When referring to itself, the custom is to use a _to concept_ of `self`. (self is the default if you don't supply a _to concept_. e.g `eating | squid | nil`
- link value: A value for the association. The default value is `nil`. Some examples:
  - `population quantity | self | 12`
  - `surface color | self | red`
  - `distance measurement | self | {"image_reference_uuid": "acc435...", "x": [100, 234], "y": [34 1200], "comment": "dorsal spine"}`
  
A final optional parameter is the mimetype of the _link value_. The default is `text/plain`, but it could `application/json` or whatever your specific application needs

### Create

In [ ]:
# Create with minimum required parameters
association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "swimming"})
show("POST: " + association_url, association)

In [ ]:
# Create with all possible parameters
association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "distance measurement", 
    "to_concept": "self",
    "link_value": '{"image_reference_uuid":' + image['image_reference_uuid'] + ', "x": [100, 234], "y": [34 1200], "comment": "dorsal spine"}',
    "mime_type": "application/json"})
show("POST: " + association_url, association)

### Update

In [ ]:
# Modify an existing association. We use all fields here but normally
# just include the ones you are changing
url = "%s/%s" % (association_url, association['uuid'])
association = put(url, data = {
    "observation_uuid": annotation["observation_uuid"],
    "link_name": "eating",
    "to_concept": "Cranchia scabra",
    "link_value": "nil",
    "mime_type": "text/plain"})
show("PUT: " + url, association)

In [ ]:
# Special method to change ALL to_concepts in the data store. Useful when you've changed
# a species name and need to update all data
url = "%s/toconcept/rename" % (association_url)
r = put(url, data = {
        "old": "Cranchia scabra",
        "new": "Taonius borealis"})
show("PUT: " + url, r)

### Find

In [ ]:
# Find by uuid
url = "%s/%s" % (association_url, association['uuid'])
association = get(url)
show("GET: " + url, association)

In [ ]:
# Find all associations in a video or image group with a given link_name
url = "%s/%s/%s" % (association_url, annotation['video_reference_uuid'], "eating")
associations = get(url)
show("GET: " + url, associations)

In [ ]:
# Count the number of usages of a particular to_concept.
url = "%s/toconcept/count/%s" % (association_url, "Taonius borealis")
r = get(url)
show("GET: " + url, r)

### Delete

In [ ]:
url = "%s/%s" % (association_url, association['uuid'])
delete(url)

## ImagedMoment API

All images and annotations must be related back to real-world time AND/OR a moment in a video. This is represented by an imaged moment which contains one or more of the following:

- timecode
- elapsed time (since the start of a video)
- recorded timestamp (the time that an image or video frame was captured

When you set one of these indicies using the Annotation or Image API the appropriate imaged moment is created (if needed) or the data will be moved to an existing imaged moment. You can __not__ create an ImagedMoment directly with this API.

### Find

In [ ]:
# Find all (by default the limit and offset are 1000 and 0 repspectively)
imaged_moments = get(imaged_moment_url)
show("GET: " + imaged_moment_url, imaged_moments)


In [ ]:
# Find all using explicit limit and offset
url = "%s?limit=2&offset=0" % (imaged_moment_url)
imaged_moments = get(url)
show("GET: " + url, imaged_moments)

In [ ]:
# Find one by its UUID
url = "%s/%s" % (imaged_moment_url, imaged_moments[0]['uuid'])
imaged_moment = get(url)
show("GET: "+ url, imaged_moment)

In [ ]:
# Find all video_reference_uuids used in the entire database
url = "%s/videoreference" % (imaged_moment_url)
vrs = get(url)
show("GET: " + url, vrs)

In [ ]:
# Find video_reference_uuids used in the database but limit with limit and offset
url = "%s/videoreference?limit=2&offset=0" % (imaged_moment_url)
vrs = get(url)
show("GET: " + url, vrs)

In [ ]:
# Find all imaged moments for a given video
url = "%s/videoreference/%s" % (imaged_moment_url, vrs[0])
imaged_moments = get(url)
show("GET: " + url, imaged_moments)

In [ ]:
# Find an imaged moment by one of the observations it contains
url = "%s/observation/%s" % (imaged_moment_url, annotation['observation_uuid'])
imaged_moment = get(url)
show("GET: " + url, imaged_moment)

### Update

In [ ]:
# We show all possible fields here, but you only need to include the ones you change
url = "%s/%s" % (imaged_moment_url, imaged_moment['uuid'])
imaged_moment = put(url, data = {
    "timecode": "11:22:33:00",
    "elapsed_time": 100,
    "recorded_timestamp": iso8601(),
    "video_reference_uuid": str(uuid.uuid4())})
show("PUT: " + url, imaged_moment)

### Delete

Be aware that deleting an imaged moment deletes it, all the observations at that moment, all image references at that moment and all ancillary data at that moment. Be sure that's really what you want to do.

In [ ]:
url = "%s/%s" % (imaged_moment_url, imaged_moment['uuid'])
delete(url)

## Observation API

Observations are the individual annotations. Thsi API has a number of find operations, an update one (although you can use the annotation api for that) and allows you to delete individual observations



In [ ]:
# House keeping. Since we're deleting stuff, let's make sure we have something to work with
# Create an new annotation
annotation = post(annotation_url,
                  data = {"video_reference_uuid": str(uuid.uuid4()),
                          "concept": "Nanomia bijuga",
                          "observer": "brian",
                          "recorded_timestamp": iso8601()})

association = post(association_url, data = {
    "observation_uuid": annotation['observation_uuid'],
    "link_name": "swimming"})

image = post(image_url, data = {
    "video_reference_uuid": annotation['video_reference_uuid'],
    "url": "http://foobar.org/awesomeimage_" + str(random.randint(0, 100000)) + ".jpg",
    "recorded_timestamp": annotation['recorded_timestamp']})


### Find

In [ ]:
# Find an observation by its uuid
url = "%s/%s" % (observation_url, annotation['observation_uuid'])
observation = get(url)
show("GET: " + url, observation)

In [ ]:
# Find observations by video
url = "%s/videoreference/%s" % (observation_url, annotation['video_reference_uuid'])
observations = get(url)
show("GET: " + url, observations)

In [ ]:
# Find by an observation by an association it contains
url = "%s/association/%s" % (observation_url, association['uuid'])
observation = get(url)
show("GET: " + url, observation)

In [ ]:
# Find all concepts used on all annotations
url = "%s/concepts" % (observation_url)
concepts = get(url)
show("GET: " + url, concepts)

In [ ]:
# Find all concepts used when annotating a particular video or image group
url = "%s/concepts/%s" % (observation_url, annotation['video_reference_uuid'])
concepts = get(url)
show("GET: " + url, concepts)

In [ ]:
# Get a count of occurences of obserations with a particular concept
url = "%s/concept/count/%s" % (observation_url, concepts[0])
n = get(url)
show("GET: " + url, n)

### Update

In [ ]:
# Update with all fields. Normally, just include the fields that you want to change
url = "%s/%s" % (observation_url, annotation['observation_uuid'])
observation = put(url, data = {
    "concept": "Teuthoidea",
    "observer": "Barack Obama",
    "observation_timestamp": iso8601(),
    "duration": 1000,
    "group": "AUV Dorado",
    "activity": "descent"})
# There's also an imaged_moment_uuid field if you want to move an observation to a different moment.
# I didn't include it because of time and more complicated setup.

show("PUT: " + url, observation)

In [ ]:
# Update all observations that use a concept, e.g. renamed species
# i.e. Globally change a concept in the database.
url = "%s/concept/rename" % (observation_url)
r = put(url, data = {
    "old": "Teuthoidea",
    "new": "Grimpoteuthis"})
show("PUT: " + url, r)

### Delete

In [ ]:
# Note that if an imaged moment only contains the observation you are deleting
# the imaged_moment will be deleted to.
delete_url = "%s/%s" % (observation_url, annotation['observation_uuid'])
delete(delete_url)

## ImageReference API

ImageReference is a pointer to an actual image. In general, you use the Image API to create and update them. You will need to use the ImageReference API to delete them though

### Update

In [ ]:
# As before we show all params you can change. But you only have to provide the ones you 
# are actually changing
url = "%s/%s" % (image_reference_url, image['image_reference_uuid'])
image_reference = put(url, data = {
    "url": "http://foobar.org/vaderimage_" + str(random.randint(0, 100000)) + ".webp",
    "format": "image/webp",
    "width_pixels": 5000,
    "height_pixels": 3000,
    "description": "Tripod-mounted camera image",
    "imaged_moment_uuid": annotation['imaged_moment_uuid']
    })
show("PUT: " + url, image_reference)

### Find

In [ ]:
url = "%s/%s" % (image_reference_url, image_reference['uuid'])
image_reference = get(url)
show("GET: " + url, image_reference)


### Delete

In [ ]:
url = "%s/%s" % (image_reference_url, image_reference['uuid'])
delete(url)